# Hands-on Lab: Build an Interactive Dashboard with Ploty Dash

Rather than using the provided theiadocker instance that does not persist data, I am adapting the assignment to be run on jupyter notebook.



### Import Required Packages

In [1]:
# Import required packages
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import wget
import webbrowser

In [2]:
import warnings
warnings.filterwarnings("ignore", message="elementwise comparison failed; returning scalar")

### Import Dataset

In [3]:
#Run the following wget command line in the terminal to download dataset as spacex_launch_dash.csv
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df=pd.read_csv(url)

In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [6]:
# Download a skeleton Dash app to be completed in this lab:
spacex_app_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py')


100% [................................................................................] 2110 / 2110

In [7]:
Launch_site=spacex_df['Launch Site'].unique()
Launch_site

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

### Dashboard Application Task

In [12]:
# skeleton Dash app to be completed in this lab, downloaded from:
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py


# Create a dash application
app = dash.Dash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                               
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                html.Div([
                                        html.Div(dcc.Dropdown(id='site-dropdown',
                                                              options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                        {'label': Launch_site[0], 'value': Launch_site[0]},
                                                                        {'label': Launch_site[1], 'value': Launch_site[1]},
                                                                        {'label': Launch_site[2], 'value': Launch_site[2]},
                                                                        {'label': Launch_site[3], 'value': Launch_site[3]},
                                                                        ],
                                                              value='ALL',
                                                              placeholder="place holder here",
                                                              searchable=True,
                                                             ),
                                                ),
                                ]),
                                

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Br(),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart'), style={'width':'65%'}),
                                html.Br(),

                                html.P("    PAYLOAD RANGE (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500:'7500',
                                                       10000:'10000',
                                                      },
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                              html.Div(dcc.Graph(id='success-payload-scatter-chart'), style={'width':'65%'}),
                                ])





# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['class']==1]
        
        fig = px.pie(filtered_df, values='class', names=('Launch Site'), 
                     title='Total Successful Launches for All Sites')
        
        fig.update_layout(legend_title='Launch Sites')
        
        return fig
    
    else:
        # return the outcomes piechart for a selected site
        
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.pie(filtered_df, names=('class'),
                 title=('Total Successful Launches for site '+ str(entered_site)))

        # Map legend labels to custom values
        legend_labels = {'0': 'Failed', '1': 'Success'}

        # Extracting labels directly from the DataFrame
        data_labels = filtered_df['class'].astype(str)

        # Update traces to set custom labels
        fig.update_traces(
            customdata=[legend_labels[key] for key in data_labels],
            legendgroup=str(data_labels.unique()[0]),  # Convert to string
            labels=[legend_labels[key] for key in data_labels] ) # Set custom legend labels)  

        # Update layout to change the legend title
        fig.update_layout(legend_title='Launch Outcomes')

        return fig

   

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback([ Output(component_id='success-payload-scatter-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])

def get_scatter(entered_site, payload_value):

    if entered_site == 'ALL':
        
        filterload_df = spacex_df[(spacex_df['Payload Mass (kg)'] < payload_value[1]) 
                                  & (spacex_df['Payload Mass (kg)'] > payload_value[0])]
        
        fig=px.scatter(filterload_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                       title='Correlation Between Payload and Success for All Sites')
        
        fig.update_layout(yaxis_title='Launch Outcome', width=800, height=320)
        fig.update_yaxes(tickvals=[0, 1], ticktext=['Failed  ', 'Success  '])
  
        
        return [fig]

    else:
        filterload_df = spacex_df[(spacex_df['Payload Mass (kg)'] < payload_value[1])
                              & (spacex_df['Payload Mass (kg)'] > payload_value[0])]
        
        filtersite_df = filterload_df[filterload_df['Launch Site'] == entered_site]
        
        fig=px.scatter(filtersite_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                   title=('Correlation Between Payload and Success for '+ str(entered_site)))
        
        fig.update_layout(yaxis_title='Launch Outcome', width=800, height=320)
        fig.update_yaxes(tickvals=[0, 1], ticktext=['Failed  ', 'Success  '])
        
        return [fig]


    
    
# Run the app
if __name__ == '__main__':
    app.run_server(port=8002, host='127.0.0.1') # to open in the notebook
    webbrowser.open_new('http://127.0.0.1:8002') ## to open in a new tab in the webbrowser

# Finding Insights Visually

> **Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:**


> **Questions**,
>
> 1. Which site has the largest successful launches?
> 2. Which site has the highest launch success rate?
> 3. Which payload range(s) has the highest launch success rate?
> 4. Which payload range(s) has the lowest launch success rate?
> 5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

> **Answers:**
>
>1. According the pie chart is KSC LC-39A with 10 successful launches
>
>2. According the pie chart is KSC LC-39A with 41.7% of success
>
>3. From 2500 to 5000 kg of payload, F9 got 11 success and 9 fails, 122% launch success rate
>
>4. From 5000 to 7500 kg of payload, F9 got 2 success and 7 fails, 29% launch success rate
>
>5. Acording the scatter plot, the FT version has the highest launch sucess rate 
